# Radial Velocity Archive Scraper

- Query [DACE](https://dace.unige.ch/DACE.php) for radial velocity measurements.


In [29]:
import os
import json
from dace_query.spectroscopy import Spectroscopy

class RadialVelocityQuery:

    def __init__(self, target=None):
        self.target = target
        self.data = {
            "target": target,
            "time": [],
            "instrument": [],
            "rv": [],
            "rv_err": []
        }

        if target is not None:
            self.query_dace()

            # more queries can be added here

    def query_dace(self):

        try:
            # query dace for number of RV observations
            observedTargets = Spectroscopy.query_database(
                limit=1000,
                filters={"public": {"is": True}, "obj_id_catname": {"contains": [self.target]}},
                output_format="pandas"
            )
        except:
            print(f"error for {self.target}")

        if len(observedTargets) == 0:
            print(f"No RV observations for {self.target}")
        else:
            print(f"Found {len(observedTargets)} RV observations for {self.target} from DACE")

            # extract time, instrument name, rv, error
            self.data["time"].extend(observedTargets["obj_date_bjd"] + 2450000) # TODO check this conversion
            self.data["instrument"].extend(observedTargets["ins_name"])
            self.data["rv"].extend(observedTargets["spectro_ccf_rv"])
            self.data["rv_err"].extend(observedTargets["spectro_ccf_rv_err"])

            # TODO add activity index? other metrics?

    ###############################
    # add more functions here
    ###############################

    def save_to_disk(self, path):
        # save data as json
        with open(path, 'w') as f:
            json.dump(self.data, f)

    @staticmethod
    def load_from_disk(path):
        # load data from json
        with open(path, 'r') as f:
            data = json.load(f)
        rv = RadialVelocityQuery(target=None)
        rv.data = data
        rv.target = data["target"]
        return rv


In [30]:
target = "WASP-77A"

rv = RadialVelocityQuery(target=target)
rv.data

Found 19 RV observations for WASP-77A from DACE


{'target': 'WASP-77A',
 'time': [2505188.6258,
  2505891.5721,
  2505914.6783,
  2505832.8615,
  2505856.7095,
  2505169.692,
  2505832.904,
  2505916.6681,
  2505170.6625,
  2505915.6775,
  2505891.7468,
  2505163.6091,
  2505890.7386,
  2505889.7458,
  2505890.537,
  2505832.911,
  2505069.7803,
  2505917.65,
  2505918.6645],
 'instrument': ['CORALIE',
  'HARPS',
  'CORALIE',
  'HARPS',
  'CORALIE',
  'CORALIE',
  'HARPS',
  'CORALIE',
  'CORALIE',
  'CORALIE',
  'HARPS',
  'CORALIE',
  'HARPS',
  'HARPS',
  'HARPS',
  'HARPS',
  'CORALIE',
  'CORALIE',
  'CORALIE'],
 'rv': [1969.6,
  1767.0,
  1710.5,
  1462.6,
  1865.5,
  1967.3,
  1511.1,
  1708.7,
  1570.6,
  1332.8,
  1988.5,
  1336.2,
  1865.6,
  1406.3,
  2022.0,
  1523.3,
  1324.6,
  1954.4,
  1554.2],
 'rv_err': [5.4,
  4.4,
  4.3,
  2.0,
  5.3,
  4.9,
  2.4,
  4.8,
  5.3,
  4.4,
  4.6,
  5.2,
  4.1,
  3.6,
  3.7,
  2.5,
  4.9,
  4.7,
  6.5]}